In [4]:
from gensim.models.keyedvectors import KeyedVectors
import os
import numpy as np
from tqdm import tqdm
import pandas as pd 


In [3]:
model =  KeyedVectors.load_word2vec_format('/Castor-data/embeddings/word2vec/GoogleNews-vectors-negative300.txt', binary=False)


In [8]:
>>> from nltk.tokenize import TweetTokenizer
>>> tknzr = TweetTokenizer()
>>> s0 = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
>>> tknzr.tokenize(s0)

['This',
 'is',
 'a',
 'cooool',
 '#dummysmiley',
 ':',
 ':-)',
 ':-P',
 '<3',
 'and',
 'some',
 'arrows',
 '<',
 '>',
 '->',
 '<--']

In [35]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [45]:
def calculate_cosine_similarity_matrix(query, document):
    query_terms = tknzr.tokenize(query)
    doc_terms = tknzr.tokenize(document)
    
    sim_mat = []
    for query_term in query_terms:
        row = []
        for doc_term in doc_terms:
            if doc_term not in model:
                doc_term = "UNK"
            if query_term not in model:
                query_term = "UNK"
            
            sim_vector = cos_sim(model[query_term], model[doc_term])
            row.append(sim_vector)
        sim_mat.append(row)
        
    # print(np.array(sim_mat))
    return sim_mat



In [50]:
f = open("/u4/w85yang/MatchZoo/data/tweets/TweetCorpus/train_qrels_2011_2012_2013_all.txt") # test_ql_2014
for l in tqdm(f):
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    directory = "/mnt/collections/w85yang/cosine/topic_doc_mat/{}".format(qid)
    out = os.path.join(directory, "{}.npy".format(docid))
    if not os.path.exists(directory):
        os.makedirs(directory)
    sim_mat = calculate_cosine_similarity_matrix(a, b)
    np.save(out, sim_mat)


55000it [00:53, 1025.84it/s]


In [5]:

def load_idf():
    df = pd.read_csv("/mnt/collections/w85yang/wikiextractor/idf_all/idf_terms.csv")
    return df.set_index('token')["idf"].to_dict()

def get_qidf(a):
    query_terms = tknzr.tokenize(a)
    qidf = []
    for w in query_terms:
        if w in idfs:
            qidf.append(idfs[w])
        else:
            qidf.append(15)
    return qidf

In [6]:
idfs = load_idf()

In [21]:
qid2idf = {}
f = open("/u4/w85yang/MatchZoo/data/tweets/TweetCorpus/test_ql_2014.txt") # test_ql_2014
for l in tqdm(f):
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    qid2idf[qid] = get_qidf(a)

55000it [00:00, 61988.15it/s]


In [13]:
# f = open("/u4/w85yang/MatchZoo/data/tweets/TweetCorpus/train_qrels_2011_2012_2013_all.txt") # test_ql_2014
f = open("/u4/w85yang/MatchZoo/data/tweets/TweetCorpus/test_ql_2014.txt")
qid_seen = set()
for l in tqdm(f):
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    directory = "/mnt/collections/w85yang/cosine/query_idf/topic_term_idf"
    out = os.path.join(directory, "{}.npy".format(qid))
    if qid in qid_seen:
        continue
    qid_seen.add(qid)
    if not os.path.exists(directory):
        os.makedirs(directory)
    np.save(out, get_qidf(a))

55000it [00:00, 263715.20it/s]


In [78]:
vs = []
for k in idfs:
    vs.append(idfs[k])

## Debug code

In [62]:
qid2docid = {}
f = open("/u4/w85yang/MatchZoo/data/tweets/TweetCorpus/train_qrels_2011_2012_2013_all.txt") # test_ql_2014
for l in tqdm(f):
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    if qid not in qid2docid:
        qid2docid[qid] = set()
    qid2docid[qid].add(docid)


0it [00:00, ?it/s]
36921it [00:00, 369205.07it/s]
78969it [00:00, 383224.48it/s]
126115it [00:00, 406020.84it/s]
179348it [00:00, 437135.31it/s]
196425it [00:00, 463063.64it/s]

In [64]:
f = open("/u4/w85yang/copacrr/data/qrels.adhoc.6y") # test_ql_2014
for l in tqdm(f):
    qid, _, docid, _ = l.replace("\n", "").split()
    if qid not in qid2docid:
        print(qid)
    else:
        if docid not in qid2docid[qid]:
            print(qid, docid)


0it [00:00, ?it/s]
6731it [00:00, 67309.10it/s]

1 34951513591783424
1 34951399884197888
1 34949898495328256
1 34946507903864832
1 34946292765425664
1 34945584968372224
1 34945186928787456
1 34944944452018176
1 34944634237095936
1 34943924992737280
1 34941068587831296
1 34940492118499328
1 34939797898272768
1 34939203464871936
1 34937962802184192
1 34937908452397056
1 34937107466162176
1 34936085628989440
1 34932398680440832
1 34931124287311872
1 34929603411709952
1 34926949637820416
1 34926022671470592
1 34925209437872128
1 34924859251228672
1 34922777454579712
1 34920407576346624
1 34919572792553472
1 34916293438087168
1 34910978344747008
1 34910401741193216
1 34903236833972224
1 34899137438687233
1 34898389988548608
1 34896068814700544
1 34890434388049920
1 34889246292582400
1 34887281231794176
1 34884927942176768
1 34882084875145216
1 34881869497503744
1 34876924790714368
1 34852790169772032
1 34851536446160897
1 34835677711433728
1 34834670793396224
1 34828591912984576
1 34795905815158784
1 34775042264928256
1 34770115417350144



11290it [00:00, 56052.40it/s]
15287it [00:00, 47976.89it/s]


8 34388820120047616
8 34388610966888450
8 34382927693496320
8 34373324561715200
8 34367653434892288
8 34361563318063106
8 34350331664007168
8 34327909254565888
8 34302036547735552
8 34272777393410048
8 34267214450270208
8 34201849716285440
8 34081157758844928
8 34070676335759360
8 34057944957976578
8 34055600639844352
8 34027706639192064
8 33987348857356288
8 33945090242117632
8 33905582880591872
8 33892980213415937
8 33884616527974400
8 33762045753040896
8 33717496494170112
8 33710526169096193
8 33674392261099520
8 33658638203760640
8 33563237563895808
8 33549749881671681
8 33537501259890688
8 33531241156317184
8 33529147561082880
8 33526364061564928
8 33523537478156288
8 33520572428333056
8 33512726160617472
8 33500057571430400
8 33361392627089408
8 33300323699265536
8 33297285500641280
8 33257280321490944
8 33184619025141760
8 33171932987793411
8 33138176272896000
8 32939116312133632
8 32824329922150401
8 32822707884130304
8 32804082171252737
8 32761735383351296
8 32576078732066816


20851it [00:00, 50036.66it/s]
26047it [00:00, 49309.90it/s]

...

216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
216
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217


219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219
219


222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222
222



262466it [00:05, 44237.37it/s]

225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225


In [51]:
docid

'308261580448075777'

In [ ]:
'29251102373314560'
'308261580448075777'
'34950344446312448'
'34951860221648896'